## Features of Good Design

Hi guys, this lecture is a bit different, today we are mostly glossing over Python itself and instead we are going to be talk about software development. 

At this moment in time you are writing tiny programs, programs that in many cases are ten lines (or less) long. And let me tell you, when try your hand at writing programs two, or three-thousand lines long you will quickly learn that not only is it significantly harder to get it working, you will also find **the skills you need change as well**. For small problems, knowledge of  syntax and language semantics are often sufficient and concepts like ‘readability’ can even be ignored (although you shouldn’t). However, if you try to scale up and write something over weeks and months you will likely find your casual disregard for readability ends up biting you. And similarly, if you jumped ‘straight-in’ without considering the ‘bigger picture’ chances are you will end up rewriting a lot of code as you start to realise, weeks later, that the current implementation is not scalable, or something. 

Today’s lecture is intended as a introduction to some of skills you will likely need once you try to develop a substantial (e.g 300+ line) programs. **The TLDR version; figuring out a good design ‘off the bat’ can potentially save you hours upon hours of work later on down the line.** 

Today I will be showing you an example of how we might code up a game of chess. But crucially I’m going to skip over a lot of the ‘low-level’ stuff and instead try to provide a ‘high-level’ sketch for what such a program may look like.  On that note, when reading today’s code please consider it a ‘rough sketch’ rather than definitive, if you look at it with a magnifying glass it will collapse like a house of cards. But as I say, today is all about the bigger picture, or  ‘high-level’ thinking. 

There is a saying in England:

>“[you] can’t see the forest for the trees”*. 

It means that if you examine something closely (i.e. each tree) you might miss the bigger picture (i.e. that all the trees come together to make a forest). Well, all of the lectures in this series have been talking about trees, and now we are talking about the forest. 

## What is good design?

So before we start looking at a chess game lets say a few words about design; in particular, what counts as good program design?

#### Simplicity

    Simple is better than complex.
    Complex is better than complicated.
    [...]
    If the implementation is hard to explain, it's a bad idea.
    If the implementation is easy to explain, it may be a good idea.
    
As always, Tim Peter's ‘Zen of Python’ has a thing or two to say about design, the lines highlighted here place an emphasis on simplicity and clarity of expression, these concepts are core to the entire language and we would do well to remember that;  if things start to get complicated maybe it would be prudent to take a step back and reconsider our approach. 

#### Performance

At first glance we might think performance is a 'low-level' consideration. You write something and then find ways to saving a byte of memory here or there. But considering performance merely as ‘fine-tuning’ would be a crucial mistake. 

Those of you that watched my 'joy of fast cars' lecture would have seen a few examples of such low-level 'fine tuning', in one example I showed how we could optimize a call to range such that we could search for prime numbers faster. And for what it is worth this tinkering did in fact pay off, we were able to lower the time taken from ten seconds to three seconds, our tweaking resulted in a performance boost upwards of 66%.

However, that lecture also contained a ‘high-level’ idea as well; our tinkering with the range function was, although faster, still blindly searching for *a needle in a haystack*. We then stepped back and wondered if there was a better way to do it; and with a little help from Google we found a way to *generate* primes efficiently rather than *searching* for them in a sea of non-primes. And the algorithm we found to was over twenty times faster than what we could with ‘low-level’ tinkering. 

The lesson here is that good design choices, even if executed poorly, can easily out-perform bad ideas implemented well. If you want to know more about this, please check do some reading on [‘algorithmic complexity’](https://en.wikipedia.org/wiki/Analysis_of_algorithms) and Big(O) notation (we wont cover this stuff in this course).

In short, good design/algorithm choices tend to be very performant once we scale up the problem to huge data sets, and this is why its worth taking the time to come up with a good idea.

#### Generalisable / Reusable

Writing good code once and then reusing it is often better than starting from scratch each time. The way to make code reusable is to generalise it to solve variety of problems. This concept is probably best understood by example. 

Suppose we were making a function that counted *1-to-100*. What can we use this for other than its intended purpose? 

Now suppose we write a function that counts from *n-to-m*. This code works for the current problem but because its design is generalised we may be able to reuse this code at a later date, in this project or the next. 

In [1]:
# One use, "throw away" code:
def one_to_one_hundred():
    for i in range(1, 101):
        print (i)
        
# Multi use, 'generalised' code:
def n_to_x(n, m):
    for i in range(n, m+1):
        print(i)

#### Beauty

> *Beautiful is better than ugly.*  
Tim Peters, ‘Zen of Python’

Beauty!? At first glance making beauty a consideration my sound like a strange or 'out-of-place' concept. But if you take a broad view of human achievement you’ll find that we mere mortals make things, and then make those things beautiful.  Just think of something like a sword, its an object made with the most brutal of applications in mind and yet we still decided that this was an object worthy of being made beautiful. 

Another discipline where discussions of aesthetics may initially seem out-of-place is mathematics, and yet, there are no shortage of mathematicians throughout the ages  discussing the [aesthetic qualities of field](http://www.huffingtonpost.com/david-h-bailey/why-mathematics-matters_b_4794617.html) and moreover there is some [experimental evidence](http://www.bbc.co.uk/news/science-environment-26151062) to suggest mathematicians genuinely see beauty in formula's in the same way the rest of us see beauty in music or art. For some, beauty truly is the joy of mathematics.

> *"Why are numbers beautiful? It's like asking why is Beethoven's Ninth Symphony beautiful. If you don't see why, someone can't tell you."*
Paul Erdos

I think it would be wrong to dismiss beauty as a trivial aspect of mathematics or programming for that matter.  There truly is a joy in experiencing good code, you just need to learn to appreciate it, I guess. 

## Building a Chess Program...

The above discussion highlighted a few aspirations we should have when trying to come up with a solution to this problem; the more beautiful, generalisable, perfomant we can make our code the better. 

Before we start coding, lets type out a list of things we need to do:

1. Represent the board (8x8 grid, alternating black/white squares)
1. Define piece movement, capture rules.
1. Define all other rules (e.g. promotion, castling, checkmate, 3-fold repetition, etc)
1. Peripherals (e.g. clocks, GUI's, multi-player features like match-making, etc)

Thats a lot of stuff to do right there, today's lecture will mostly deal with points one & two.

### Building the board

How should we represent a board in Python? This question mostly just reduces to what data-type we should use. Right now, I have two candidates in mind; strings and lists. 

We could of course jump ‘straight-in’, pick one the data types at random and see what happens but, as alluded to in the above discussions such a method is both silly and wasteful. A better use of time would be to carefully consider our options BEFORE we write even a single line of code.

#### The Board as a string

Okay, so let’s consider using a string for the board. What might that look like?

Well, the letters "QKPN" could represent the pieces (lower-case for white), and we could use the new-line escape character ("\\n") to separate the rows. Something like this:

In [2]:
print("RNBQKBNR\nPPPPPPPP\n-x-x-x-x\nx-x-x-x-\n-x-x-x-x\npppppppp\nrnbkqbnr") # 'x' and '-' represent black and white.

RNBQKBNR
PPPPPPPP
-x-x-x-x
x-x-x-x-
-x-x-x-x
pppppppp
rnbkqbnr


Actually, we can do even better than this, Python strings support unicode and there are [unicode characters for chessmen](https://en.wikipedia.org/wiki/Chess_symbols_in_Unicode#Unicode_codepoints_and_HTML). So now our string implementation even comes with some basic graphics:

In [3]:
print("♙♙♙♙♙♙♙♙\n♖♘♗♔♕♗♘♖\n")

♙♙♙♙♙♙♙♙
♖♘♗♔♕♗♘♖



Okay so the board as a string seems possible, but are there any drawbacks of an implementation like this? Well, remember that strings are an *immutable data-type*, which means that every time we want to change the board we have to make a new one. Not only would this be computationally inefficient it may also be a bit ‘fiddly’ to actually change the board.

For example, lets see what sort of work we would have to to make the move 1.Nf3:

In [4]:
def make_move(move):
    """Takes a string a returns a new string with the specified move"""
    # Code here
    pass

# Our new function would have to take the original string and return the new string (both below)...
original_string = "RNBQKBNR\nPPPPPPPP\n-x-x-x-x\nx-x-x-x-\n-x-x-x-x\npppppppp\nrnbkqbnr"
new_string =      "RNBQKBNR\nPPPPPPPP\n-x-x-x-x\nx-x-x-x-\n-x-x-n-x\npppppppp\nrnbkqb-r"
print(new_string)

RNBQKBNR
PPPPPPPP
-x-x-x-x
x-x-x-x-
-x-x-n-x
pppppppp
rnbkqb-r


Now, to be clear, it is certainly possible to make the “make_move” function work, but it does seem to have several small moving parts and therefore probably lots of interesting ways to go wrong. And then lets think about the more complex functions; if movement seems a bit tricky, how easy do we think defining checkmate is going to be?

Basically, using strings seems doable but complicated. And as Tim Peters says, simple and complex are both better than complicated. Alright, on that note, let’s see if lists seem more straight-forward.

#### The Board as a List

       [[00, 01, 02],
        [10, 11, 12],
        [20, 21, 22]]
        
The above is a nested list but we have put each of the sublists on a new line to make it easier to visualise how such a structure can work like a game board. The numbers represent the (x, y) coordinate of that 'square'. And remember that lists can contain strings as well, so this option doesn't stop us from using those pretty graphics either.  

Compared to the string implementation, the 1.Nf3 move should be somewhat straight-foward:
    
    Current_square  = {Black or White}
    New_square      = {Knight}

Current and New_squares being represented as something like 'board[x][y]'. At first glance, this seems considerably easier than a messing arround mutating strings. 

There is also another possible advantage to lists as well, and that is they can store a variety of data-types. I haven't spoken about classes in this lecture series and I'm not going to into detail * (classes are not suitable for a beginner class, in my opinion)* But, I’ll will very briefly introduce you to the concept and how we could use it here. 

Basically  Python makes it possible to create your own objects with their own methods. Yes, that’s right, we could *literally* make a “Knight” and give it the method ‘move to’. And yes, we could call the method just like any list or string method we have seen. 

Using classes then, it is, in some sense, *literally* possible to put a knight on a square on the board. Below I’ve provided a very rough sketch of what such a class could look like. Don’t bother running it though, it wont work because I’ve left out all the necessary bits of ‘low-level’ code.

In [5]:
class Knight (object):
    
    def __init__ (self, current_square, colour):
        
        # define what this knight 'is', its properties etc
        pass
    
    def legal_moves(square1, square2):
        """Checks if moving from square1 to square2 is a legal move for the peice.
        Returns True/False"""
        # Code goes here...
        pass
    
    def move_to(self, square):
        if legal_moves(current_square):
            # Move to a square.
            pass
    
    # Other methods goe here. 

# And once you have made the knight class we can make a knight. 
a_knight = Knight(square, "White")

# Once the knight is made, we can move it using the move_to method:
a_knight.move_to(new_square)

NameError: name 'square' is not defined

## Defining Piece movement

Alright, onto the next problem. "How are we going to make the peices move?" And once again a smart choice here will make things so much easier than it otherwise could be.

One simple approach is to write a function for each piece, like so:

In [ ]:
def bishop(position, board=""):
    """ When given a starting square, returns all squares the bishop can move to"""
    pass
    
def rook(position, board=""):
    """ When given a starting square, returns all squares the rook can move to"""
    pass
    
def queen(position, board=""):
    """When given a starting square, returns all squares the queen can move to"""
    pass

At first glance this code seems pretty good, but there are a few drawbacks. Firstly it looks like we are going to be repeating ourselves a lot; queen movement for example is just a copy & paste of the rook + bishop. The king function is likely a copy & paste of queen but where we change the distance to 1 and add a bunch of extra rules for check, and so on. 

And by the way guys, repeating oneself is NOT quite the same as reusing code! 

The second issue is that if we define movement in terms of an 8x8 board we lack generalisation of the problem.  How can we do better? Well, how about we define movement patterns and then define the pieces in terms of movement. Here, let me show you:

In [ ]:
def diagonal(position, direction="frontleft, rearright, etc", distance =1):
    """Returns all possible square in diagonal direction of n distance from original position"""
    # code here
    pass

def row_movement(position, direction="left/right", distance=1):
    """
    >>> row_movement( (2, 2), direction=Left, distance =2)
    [(2,2), (1, 2), (0, 2)]
    >>> row_movement( (2, 2), direction=right, distance= 4)
    [(2, 2), (3,2), (4,2), (5,2), (6,2)]
    """
    
    possible_moves = []
    
    if direction == left:
        for move in range(distance):
            possible_moves.append ( (x - move) )

def col_movement(position, direction="up/down", distance=1):
    # code here
    pass

# ...and so on...

Notice here that we have defined movement without reference to a board, our code here simply takes an (x,y) coordinate in space and will keep returning valid squares until it reaches the limit set by distance. The documentation in row movement explains the idea.

With this generalisation, we should be able to handle larger or even rectangular boards. And now we can define pieces with just a few lines of code AND without repeating ourselves. Like so:

In [ ]:
def queen_movement(position):
    possible_moves = [diagonal(position, direction="all", distance="infinite"),
                      row_movement(position, direction="all", distance="infinite")
                      col_movement(postion, direction="all", distance="infinite")]
    # ...and so on... 
    
def pawn_movement(position):
    
    possible_moves = []
    
    if position == starting_position: # pawns move 2 from thier starting square.
        col_movement(position, direction="up", distance = 2)
    else:
        col_movement(position, direction="up", distance = 1)
    
    if enemy_in_attack_range # Pawns attack diagonally but move forward/back.
        diagonal(position, direction="forwardleft/right", distance= 1)
        
    possible_moves = [col_movement, diagonal]
    
    # .. and so on... 

So why might something like this be a good design choice? Well, to see the power of the this representation I think we we have to ‘zoom out’ a bit.

Because pieces are defined in terms of general movement patterns we could easily invent new pieces with this code. And, as noted above, since we do not reference a board, we have given ourselves the ability to play chess on different size and shape boards. 

This means that if in a years time we want to write a checkers, shogi, or a capablanca chess program then we have already written a bunch of useful code for that project. Neat huh? 

To see another advantage lets think about some of the other rules in chess, castling, for example. Does this 'movement first' representation make the implementation of these functions easier? Perhaps...

In [ ]:
def kingside_castling(piece_1, peice_2):
    """where "peice" is a tuple/class containing (position, type, colour, other relevant facts...)"""
    
    # check pieces are king, rook, of same colour, in the right position
    # check king is allowed to castle (e.g hasn't moved this game)
    
    # "move_peice" being some function defined elsewhere that moves a peice
    move_peice(row_movement(king, direction="left", distance=2))
    move_peice(row_movement(Rook, dierection="right", distance=2))
    
    # ... and so on ...       

The thing to note here is that although we would have to code up specific checks to see if castling is legal, the actual castling itself is fairly easily handled by the 'row function' outlined above. Had we have gone with the individual piece representation we would have probably of been forced to write special if clauses within the King/Rook functions respectively.  

But with this implementation, the special movement rule of castling is handled separately from normal movement. This should make our code easier to debug at a later date. 

Again, lets continue to dive deeper. How might check, checkmate, and statemate be defined given this implementation...

In [ ]:
def possible_moves(piece, position, friendly_position_list):
    """ Returns all possible movement squares for given piece type at given position"""
    if peice == pawn:
        possible_moves = pawn_movement(position) # 'substract' all the squares other friendly units occupty/block
    elif peice == queen:
        pass
    # ... and so on ...   
    return (piece, moves)
    
def is_check(king_position, enemies = [enemy_piece, position] )
    
    # enemies would be a list of all remaining enemy peices; we store the peice type and its position
    # eg, [ (Queen, b5), (Rook, a5), (pawn, d4), (pawn, e6), ... ] 

    for (enemy_piece, position) in enemies:
        if king_position in possible_moves(enemy_peice, position):
            return True 
    return False        

Now that we have a ‘check’ function it seems like both mate and stalemate shouldn't be too hard to implement. In the case of stalemate, all we have to do is check if:

* all possible king moves result in check (but the king is not currently in check)
* no other move exists. (e.g our a3 pawn, cannot move, for example)

What might that look like in code?

In [ ]:
def is_stalemate(king position, enemies):
    
    # King in check? If yes, break
    if not is_check(king_postion, enemies):
        # Get all friendly moves...
        for (move, piece) in possible_moves(All_friendlies, position):
            # Check if all friendly moves result in check...
            if not is_check(king, (move, piece) ):
                return False
        return True
    return False

## Conclusion

So today we moved away from the nitty-gritty and focused on the ‘big picture’. We looked at a few ways to represent a chess program in Python and although my code today was very ‘loose’ hopefully you guys the followed along and understood the main lesson I was trying to teach; **good, thought-out design matters; it makes coding faster, less frustrating, and also more expressive.**

As we move toward the final project you would do well to remember some of these principles. Alright that’s it for this lecture, no homework this week. See ya next time!